# NOAAs weather API 

In this project I am working with National Oceanic and Atmomspheric Administration (NOAAs) weather API. This API allows me to retrieve a variety of data from a specific weather station(s), of my choice.

API Documentation: https://www.ncdc.noaa.gov/cdo-web/webservices/v2#gettingStarted

Following the instructions at https://www.ncdc.noaa.gov/cdo-web/token to register for the credentials.

### Project tasks

Using the NOAA API, I am going to retrieve data for a weather station of my choice. 
   * Determine an appropriate dataset 
   * Determine an appropriatedataset type
   * Pull at least 3 years worth of data.<br>
     Note: if you pick an annual dataset, you will need to pull at least 25 years worth of data.
   * Organize your results into a meaningful representation
   * Store your result in one of the following formats:
      - csv file
      - json file
      - relational database

In [1]:
# my credentials for NOAA API. 
my_token = 'MYrltvSanupYLUQfHmtWaIHrIApRmkec'

In [2]:
my_token

'MYrltvSanupYLUQfHmtWaIHrIApRmkec'

Now we need to determine a weather station that we would like to retrieve our data for. Use the following link to get the id for a NOAA weather station. https://www.ncdc.noaa.gov/cdo-web/datatools/findstation

 I used this information to get the data set:
   * Location: Virginia Beach, VA
   * Dataset: Daily Summaries
   * Data Range: 2016-01-01 to 2019-12-01
   * Data Category: Air Temperature


From the Find A Station results, we will need to capture the following details:
   * Capture the values within the 'Network' and 'Id' fields (second cell from top, split on ':')

In [3]:
# variables based on my station search
network = 'GHCND'
ID = 'USW00013737'

# station_id = network:ID
station_id = network + ':' + ID
print(station_id)

GHCND:USW00013737


### What type of data are we looking for?
At this point we need to determine what type of data we want to retrieve. We can actually use the NOAAs API to help determine what is available for this station.

One of the documentation pages https://www.ncdc.noaa.gov/cdo-web/webservices/v2#dataTypes shows us how to query for the available datatypes for the station we have chosen above.

In [4]:
import requests
import json

# building the parameter dictionary
data = {}
data = {'limit':'1000', 'datasetid': network, 'station_id': station_id}

# calling NOAA API to get the available datatypes for this specific station
r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/datatypes',params = data, headers = {'token':my_token})

Now we need to convert the JSON output from the request to something more readable

In [5]:
# JSON to dictionary
datatypes_dict = json.loads(r.text)

# need the keys from this dictionary
datatypes_dict.keys()


dict_keys(['metadata', 'results'])

I'm going to guess that the information we are after is stored in the results key.  Let's look at the first 5 and see if we might be right

In [6]:
datatypes_dict['results'][:5]

[{'mindate': '1994-03-19',
  'maxdate': '1996-05-28',
  'name': 'Average cloudiness midnight to midnight from 30-second ceilometer data',
  'datacoverage': 1,
  'id': 'ACMC'},
 {'mindate': '1965-01-01',
  'maxdate': '2005-12-31',
  'name': 'Average cloudiness midnight to midnight from manual observations',
  'datacoverage': 1,
  'id': 'ACMH'},
 {'mindate': '1994-02-01',
  'maxdate': '1996-05-28',
  'name': 'Average cloudiness sunrise to sunset from 30-second ceilometer data',
  'datacoverage': 1,
  'id': 'ACSC'},
 {'mindate': '1965-01-01',
  'maxdate': '2005-12-31',
  'name': 'Average cloudiness sunrise to sunset from manual observations',
  'datacoverage': 1,
  'id': 'ACSH'},
 {'mindate': '1982-01-01',
  'maxdate': '2019-12-12',
  'name': 'Average wind speed',
  'datacoverage': 1,
  'id': 'AWND'}]

So, the results appear to be a list of dictionaries. 

### Determine an appropriate dataset type: 
I chose to retreive datatype set of TMIN, which is minimal daily temperature in Norfolk, VA from 2016-01-01 to 2019-12-01.

In [8]:

# append additional parameters to data dictionary
new_list = []
start_dt = 2016
years = 3

for x in range(start_dt, start_dt + years):
    data = {}
    data = {'limit':'1000', 'datasetid': network, 'stationid': station_id}
    data.update({'datatypeid': 'TMIN'})
    data.update({'units':'standard'})
    data.update({'startdate': str(x) + '-01-01'})
    data.update({'enddate': str(x) + '-12-01'})
    
    # make the request to get our year of data
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data',params = data, headers = {'token':my_token})

    #load the api response as a json
    min_temp = json.loads(r.text)
    new_list.append(min_temp)
    
new_list

[{'metadata': {'resultset': {'offset': 1, 'count': 336, 'limit': 1000}},
  'results': [{'date': '2016-01-01T00:00:00',
    'datatype': 'TMIN',
    'station': 'GHCND:USW00013737',
    'attributes': ',,W,2400',
    'value': 46.0},
   {'date': '2016-01-02T00:00:00',
    'datatype': 'TMIN',
    'station': 'GHCND:USW00013737',
    'attributes': ',,W,2400',
    'value': 36.0},
   {'date': '2016-01-03T00:00:00',
    'datatype': 'TMIN',
    'station': 'GHCND:USW00013737',
    'attributes': ',,W,2400',
    'value': 35.0},
   {'date': '2016-01-04T00:00:00',
    'datatype': 'TMIN',
    'station': 'GHCND:USW00013737',
    'attributes': ',,W,2400',
    'value': 31.0},
   {'date': '2016-01-05T00:00:00',
    'datatype': 'TMIN',
    'station': 'GHCND:USW00013737',
    'attributes': ',,W,2400',
    'value': 23.0},
   {'date': '2016-01-06T00:00:00',
    'datatype': 'TMIN',
    'station': 'GHCND:USW00013737',
    'attributes': ',,W,2400',
    'value': 33.0},
   {'date': '2016-01-07T00:00:00',
    'dataty

In [53]:
#storing data in csv
import csv

with open('CSV_assig_7.csv', 'w') as f:
    for lines in new_list:
        f.write(str(lines))